In [1]:
import pandas as pd
import pyspark
import itertools
from sklearn.utils import shuffle
frac = 0.001

In [2]:
postulaciones = pd.read_csv('Sources/fiuba_4_postulaciones_concat.csv')
test = pd.DataFrame()
postulaciones = postulaciones.drop_duplicates(['idaviso','idpostulante'])
postulantes = pd.concat([pd.read_csv('Sources/test_final_100k.csv'), pd.read_csv('Sources/fiuba_4_postulaciones_concat.csv')])
postulantes = postulantes.drop_duplicates('idpostulante')['idpostulante']
avisos = pd.read_csv('Sources/fiuba_6_avisos_detalle_limpio.csv')['idaviso']

In [3]:
print('postulantes ',postulantes.sample(frac=frac).shape[0])
print('avisos',avisos.sample(frac=frac).shape[0])

postulantes  367
avisos 27


In [4]:
i = 0
print(postulaciones.shape[0],2*postulaciones.shape[0])
while test.shape[0] < 2*postulaciones.shape[0]:
    while test.shape[0] < 2*postulaciones.shape[0]:
        aux = []
        i = i + 1
        postulantes_sample = postulantes.sample(frac=frac)
        avisos_sample = avisos.sample(frac=frac)
        for x in itertools.product(avisos_sample,postulantes_sample):
            aux.append(x)
        aux = pd.DataFrame.from_records(aux)
        aux.rename({0:'idaviso',1:'idpostulante'}, axis=1, inplace=True)
        test = pd.concat([test,aux])
    print('Antes de la poda:', test.shape[0])
    test = test.drop_duplicates(['idaviso','idpostulante'])
    print('Despues de la poda:', test.shape[0])
print(i, test.shape[0],postulaciones.shape[0])

6603752 13207504
Antes de la poda: 13208697
Despues de la poda: 13199851
Antes de la poda: 13209760
Despues de la poda: 13209738
1334 13209738 6603752


In [5]:
postulaciones.drop(['Unnamed: 0','fechapostulacion'], axis=1, inplace=True)

In [6]:
postulaciones['sepostulo'] = 1

In [7]:
postulaciones.shape

(6603752, 3)

In [8]:
postulaciones = pd.concat([postulaciones,test])

In [9]:
postulaciones.shape

(19813490, 3)

In [10]:
postulaciones = postulaciones.drop_duplicates(['idaviso','idpostulante'])
postulaciones['sepostulo'].fillna(0,inplace=True)

In [11]:
postulaciones.shape

(19804652, 3)

In [12]:
test = pd.read_csv('Sources/test_final_100k.csv').drop('id', axis=1)
postualciones = pd.concat([test,postulaciones])
postualciones = postulaciones.drop_duplicates(['idaviso','idpostulante'])
postulaciones = postulaciones[100000:]
postulaciones.shape

(19704652, 3)

In [13]:
postualciones.shape

(19804652, 3)

In [14]:
postulaciones = shuffle(postulaciones)

In [15]:
postulaciones.head()

,idaviso,idpostulante,sepostulo
5231,1.112295e+09,KBe4pdq,0.0
1813,1.112235e+09,jkVWeD0,0.0
111054,1.112293e+09,NKaQJx,1.0
6353,1.112290e+09,8MBPQkO,0.0
7881,1.111847e+09,DrXNG2A,0.0


In [16]:
postulaciones.to_csv('Sources/postulaciones_for_training.csv')